In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
from operator import attrgetter
import datetime as dt
import matplotlib.colors as mcolors
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
data=pd.read_csv('../input/ecommerce-uk-retailer/Ecommerce - UK Retailer.csv', encoding= 'unicode_escape')
data.head()

## Data Cleaning

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

Since customer ID is absolutely needed and we don't have a reliable way to replace description, we'll drop the data points having NaN in both

In [ ]:
data.dropna(subset=['CustomerID'], inplace=True)

In [ ]:
data.dropna(subset=['Description'], inplace=True)

In [ ]:
data.shape

## EDA

Questions to be answered

Q- What was the average quantity of items ordered overall?

Q- What is the average unit price of each item

In [ ]:
data.describe().transpose()

Ans1- 12 approx.

Ans2- 3.5

From the table above, it can be seen that there are orders with negative quantity — most likely returns

Lets inspect the purchases with returns

In [ ]:
# make a separate dataframe for 
dfReturns = data[data['Quantity'] < 0]
dfReturns.head()

In [ ]:
print(f'There are {len(dfReturns)} transactions for the returns')

In [ ]:
# checking the countries that have most number of returns
dfReturns['Country'].value_counts()

In [ ]:
#Q- top 10 products that are most returned?
dfReturns['Description'].value_counts().head(10)

In [ ]:
data=data[data['Quantity']>=0]
data.shape

By removing the returns, we create a bias in the dataset.

This way the initial order is taken into account even though in theory it was not realized and did not generate revenue as it was returned later on. 

Creating a 'Total Prices' Column for each purchase

In [ ]:
data["TotalPrice"] = data["Quantity"] * data["UnitPrice"]

Q- How many customers are repeat customers?

In [ ]:
order_num = data.groupby(['CustomerID'])[['InvoiceNo']].nunique().apply(display)

As we can see, there are 4339 total customers. Let's see the number of repeat customers i.e. those who made >1 order

In [ ]:
mul_orders = pd.DataFrame(data.groupby(['CustomerID'])[['InvoiceNo']].nunique())
mul_orders['InvoiceNo']=mul_orders['InvoiceNo'].astype(int)
mul_orders=mul_orders[mul_orders['InvoiceNo']>1]
len(mul_orders)

As we can see, 2845 customers made a repeat purchase. That means, around 65% customers were retained.

Q- Who are the most frequent shoppers?


In [ ]:
data["CustomerID"].value_counts().head()


Q-Number of unique products?

In [ ]:
data["StockCode"].nunique()

Q-Top 5 products bought?

In [ ]:
data["Description"].value_counts().head()

Q-Which countries placed the most orders?

In [ ]:
data['Country'].value_counts().head()

Q-Which countries spent the most?

In [ ]:

data.groupby("Country").agg({"TotalPrice": "sum"}).sort_values(by = "TotalPrice", ascending = False).head()

Q-Customers who spent the most?


In [ ]:

data.groupby("InvoiceNo").agg({"TotalPrice": "sum"}).sort_values(by = "TotalPrice", ascending = False).head()

Q-Percentage of consumers ordered more than once?

Finding Invoice month & year from invoice date

In [ ]:
data['InvoiceDate']=pd.to_datetime(data['InvoiceDate'])
def get_month(x):
    return dt.datetime(x.year, x.month, 1)
  
# Create the invoicemonth period column
data['InvoiceMonthYear'] = data['InvoiceDate'].apply(get_month)
data.head()

Q- What was the time for which each customer was retained? What is the trend of retention?

In [ ]:
grouping = data.groupby('CustomerID')['InvoiceMonthYear']
#finding and assigning earliest date of joining for each customer, here cohort refers to the time of joining
data['CohortMonthYear'] = grouping.transform('min')
data

Finding the time for which each customer stayed with the company(in months)

In [ ]:
data['InvoiceMonthYear']=pd.to_datetime(data['InvoiceMonthYear'])
data['CohortMonthYear']=pd.to_datetime(data['CohortMonthYear'])

  
invoice_year, invoice_month = data['InvoiceMonthYear'].dt.year,data['InvoiceMonthYear'].dt.month
cohort_year, cohort_month = data['CohortMonthYear'].dt.year,data['CohortMonthYear'].dt.month
years_diff = invoice_year - cohort_year
months_diff = invoice_month - cohort_month
data['MonthsRetained'] = years_diff * 12 + months_diff + 1
data.head()

Plotting the results

In [ ]:
sns.countplot(data['MonthsRetained'])
plt.xlabel("Number of months of retention")
plt.ylabel("Number of customers")
plt.show()

As we can see, a large chunk of customers stayed only for 1 month. 

We can see that there was a significant decrease in the number of customers using the product in the next month. and after that month, there is steady flow of customers for the rest of the year except that next year where less loyal customers were present. 

This could be many reasons:
1. No end of the year sales or discounts.
2. No seasonal or festival offers
(This may vary from country to country)
3. No new updates on products or interface of the store.

Going deeper, let's see the retention rate of customers cohortwise.

In [ ]:
group = data.groupby(['CohortMonthYear', 'MonthsRetained'])
cohorts = group['CustomerID'].apply(pd.Series.nunique).reset_index()
cohort_counts = cohorts.pivot(index='CohortMonthYear', columns='MonthsRetained', values='CustomerID')
cohort_sizes = cohort_counts.iloc[:,0]
retention_percent = cohort_counts.divide(cohort_sizes, axis=0)*100

In [ ]:
month_list = retention_percent.reset_index()['CohortMonthYear']

def get_month_name(x):
   return dt.datetime.strftime(x, '%b-%y')
  
month_list = month_list.apply(get_month_name)

In [ ]:

plt.figure(figsize=(15,7))
plt.title('Percentage Retention by Cohort')
sns.heatmap(data=retention_percent,annot = True,vmin = 0.0, cmap="BuPu",vmax = list(retention_percent.max().sort_values(ascending = False))[1]+3,fmt = '.1f',yticklabels=month_list)
plt.show()

As we can see, retention of customers shows a general decreasing trend as the months go by.

Based on the cohort analysis graph of Percentage Retention of cohorts, we deduce following observations:

1. Customers who purchased first time in the month of december 2010, there was significant decline in customers purchasing in the next 11 months but at the end of the year the customers came back to purchase the products in the store by a huge margin - This is due to end of the year sales or discounts done to attract new and old customers to purchase which is good but a temporary solution to increase revenue 

2. During May to August 2011, we can see that the retention rate increases after a lag. For most of the regions, the retentiaon rate increases after 8 to 10 periods but later end of year declined. The increase could be the email marketing done with the old customers that allowed them to return purchasing products but later at the end, less customers returned. due to no or less communication with the store. The store should initiate discount vaouchers for customers, who do purchases each month. This way, they will be able to increase their revenue.



Q- What is the average spending behaviour of each cohort

In [ ]:
gp=group['UnitPrice'].mean().reset_index()
average_price = gp.pivot(index='CohortMonthYear', columns='MonthsRetained', values='UnitPrice')
average_price.index = average_price.index.date

In [ ]:
plt.figure(figsize=(15,7))
sns.heatmap(data = average_price,annot=True,vmin = 0.0, cmap="BuPu",vmax = list(average_price.max().sort_values(ascending = False))[1]+3,fmt = '.1f',yticklabels=month_list)
plt.title('Average Spending by Cohort')
plt.show()

Average spending of each cohort by month remains rather stable, with some exceptions.

Q - What is the average quantity by each monthly cohort?

In [ ]:
gp=group['Quantity'].mean().reset_index()
average_quantity = gp.pivot(index='CohortMonthYear', columns='MonthsRetained', values='Quantity')
average_quantity.index = average_quantity.index.date

plt.figure(figsize=(15,7))
sns.heatmap(data = average_quantity,annot=True,vmin = 0.0, cmap="BuPu",vmax = list(average_quantity.max().sort_values(ascending = False))[1]+3,fmt = '.1f',yticklabels=month_list)
plt.title('Average Quantity by Cohort')
plt.show()

Based on the graph above, we notice that cohort on May 2011, there is an significant increase in quantity of sales at the end of the year. This may be due to the store's clearance sale that allowed them to sell the products at cheap rate to clear the stock. But By looking at the cohort retention chart in the same may 2011 cohort, it didnt improve the number of new customers arriving to the store. It may have increased the revenue but compromised the quality of the product which in turn had negative feedback from the loyal customers.

What is the trend of customer retention in each country?

In [ ]:

plt.title("Trend of customer retention by country", fontsize=18, y=1)

for x in data['Country'].unique():
  data2=data[data['Country']==x]
  sns.countplot(data2['MonthsRetained'])
  plt.xlabel(x)
  plt.show()

In a large number of countries, we observe that retention is much more compared to UK but it is overriden by the low number of customers. Hence one solution would be to focus on increasing the reach of the company in foreign markets, especially those with low attrition rates, like Netherlands,Iceland,EIRE,Australia etc.

In addition to that, there must be flexibility of having language compatibility in the online store so that customer in different countries can establish a long term relationship with the store and conduct purchases with ease. Increasing the retention rate at the same time.

Key Observations:

1. In EIRE, no cohort is formed after December, 2010. This is a major point of concern. EIRE is their fourth largest potential market. Additionally, the retention rate of EIRE is better than its counter parts reaching upto 100% after certain periods. This hints that the customer base of EIRE is very loyal. The ecommerce store should do campaigns in EIRE to further tap this potential market and gain more loyal customers, in turn, increases revenue.

2. UK market is their largest customer base. But, most of the returns are also from UK. This is an alarming situation. The store should focus more on the needs and the wants of UK  customer base to increase the revenue even further.



### Countrywise retention matrix analysis
As we observed, people from were many countries were customers. Let's look at the situation countrywise.

### UK

In [ ]:
# filtering records for UK only
dfUK = data[data['Country']=='United Kingdom']

# drop irrelevant columns and duplicate records
dfUK = dfUK[['CustomerID', 'InvoiceNo', 'InvoiceDate']].drop_duplicates()

# order month
dfUK['order_month'] = dfUK['InvoiceDate'].dt.to_period('M')

# first purchase date for that customer
dfUK['cohort'] = dfUK.groupby('CustomerID')['InvoiceDate'] \
                 .transform('min') \
                 .dt.to_period('M')

# a dataframe with cohort, order_month, n_customers, and period number
df_cohort = dfUK.groupby(['cohort', 'order_month']) \
              .agg(n_customers=('CustomerID', 'nunique')) \
              .reset_index(drop=False)
df_cohort['period_number'] = (df_cohort.order_month - df_cohort.cohort).apply(attrgetter('n'))

# pivot the df_cohort
cohort_pivot = df_cohort.pivot_table(index = 'cohort',
                                     columns = 'period_number',
                                     values = 'n_customers')

# calculate retention matrix
cohort_size = cohort_pivot.iloc[:,0]
retention_matrix = cohort_pivot.divide(cohort_size, axis = 0)


# plot the retenton matrix
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(12, 8), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(retention_matrix, 
                mask=retention_matrix.isnull(), 
                annot=True, 
                fmt='.0%', 
                cmap='RdYlGn', 
                ax=ax[1])
    ax[1].set_title('Monthly Cohorts: User Retention in UK', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # cohort size
    cohort_size_df = pd.DataFrame(cohort_size).rename(columns={0: 'cohort_size'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_df, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap=white_cmap, 
                ax=ax[0])

    fig.tight_layout()

In [ ]:
plt.plot(retention_matrix.mean())
plt.xlabel('# of periods')
plt.ylabel('retention rate')
plt.title('retention rate in the UK')

### Germany

In [ ]:
# filtering records for Germany only
dfGermany = data[data['Country']=='Germany']

# drop irrelevant columns and duplicate records
dfGermany = dfGermany[['CustomerID', 'InvoiceNo', 'InvoiceDate']].drop_duplicates()

# order month
dfGermany['order_month'] = dfGermany['InvoiceDate'].dt.to_period('M')

# first purchase date for that customer
dfGermany['cohort'] = dfGermany.groupby('CustomerID')['InvoiceDate'] \
                 .transform('min') \
                 .dt.to_period('M')

# a dataframe with cohort, order_month, n_customers, and period number
df_cohort = dfGermany.groupby(['cohort', 'order_month']) \
              .agg(n_customers=('CustomerID', 'nunique')) \
              .reset_index(drop=False)

df_cohort['period_number'] = (df_cohort.order_month - df_cohort.cohort).apply(attrgetter('n'))

# pivot the df_cohort
cohort_pivot = df_cohort.pivot_table(index = 'cohort',
                                     columns = 'period_number',
                                     values = 'n_customers')

# calculate retention matrix
cohort_size = cohort_pivot.iloc[:,0]
retention_matrix = cohort_pivot.divide(cohort_size, axis = 0)


# plot the retenton matrix
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(12, 8), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(retention_matrix, 
                mask=retention_matrix.isnull(), 
                annot=True, 
                fmt='.0%', 
                cmap='RdYlGn', 
                ax=ax[1])
    ax[1].set_title('Monthly Cohorts: User Retention in Germany', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # cohort size
    cohort_size_df = pd.DataFrame(cohort_size).rename(columns={0: 'cohort_size'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_df, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap=white_cmap, 
                ax=ax[0])

    fig.tight_layout()

In [ ]:
# plot retention rate
plt.plot(retention_matrix.mean())
plt.xlabel('# of periods')
plt.ylabel('retention rate')
plt.title('retention rate in Germany')

### Australia

In [ ]:
# filtering records for Australia only
dfAustralia = data[data['Country']=='Australia']

# drop irrelevant columns and duplicate records
dfAustralia = dfAustralia[['CustomerID', 'InvoiceNo', 'InvoiceDate']].drop_duplicates()

# order month
dfAustralia['order_month'] = dfAustralia['InvoiceDate'].dt.to_period('M')

# first purchase date for that customer
dfAustralia['cohort'] = dfAustralia.groupby('CustomerID')['InvoiceDate'] \
                 .transform('min') \
                 .dt.to_period('M')

# a dataframe with cohort, order_month, n_customers, and period number
df_cohort = dfAustralia.groupby(['cohort', 'order_month']) \
              .agg(n_customers=('CustomerID', 'nunique')) \
              .reset_index(drop=False)

df_cohort['period_number'] = (df_cohort.order_month - df_cohort.cohort).apply(attrgetter('n'))

# pivot the df_cohort
cohort_pivot = df_cohort.pivot_table(index = 'cohort',
                                     columns = 'period_number',
                                     values = 'n_customers')

# calculate retention matrix
cohort_size = cohort_pivot.iloc[:,0]
retention_matrix = cohort_pivot.divide(cohort_size, axis = 0)


# plot the retenton matrix
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(12, 8), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(retention_matrix, 
                mask=retention_matrix.isnull(), 
                annot=True, 
                fmt='.0%', 
                cmap='RdYlGn', 
                ax=ax[1])
    ax[1].set_title('Monthly Cohorts: User Retention in Australia', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # cohort size
    cohort_size_df = pd.DataFrame(cohort_size).rename(columns={0: 'cohort_size'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_df, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap=white_cmap, 
                ax=ax[0])

    fig.tight_layout()

In [ ]:
# plot retention rate
plt.plot(retention_matrix.mean())
plt.xlabel('# of periods')
plt.ylabel('retention rate')
plt.title('retention rate in Australia')

### Netherlands

In [ ]:
# filtering records for Netherlands only
dfNetherlands = data[data['Country']=='Netherlands']

# drop irrelevant columns and duplicate records
dfNetherlands = dfNetherlands[['CustomerID', 'InvoiceNo', 'InvoiceDate']].drop_duplicates()

# order month
dfNetherlands['order_month'] = dfNetherlands['InvoiceDate'].dt.to_period('M')

# first purchase date for that customer
dfNetherlands['cohort'] = dfNetherlands.groupby('CustomerID')['InvoiceDate'] \
                 .transform('min') \
                 .dt.to_period('M')

# a dataframe with cohort, order_month, n_customers, and period number
df_cohort = dfNetherlands.groupby(['cohort', 'order_month']) \
              .agg(n_customers=('CustomerID', 'nunique')) \
              .reset_index(drop=False)

df_cohort['period_number'] = (df_cohort.order_month - df_cohort.cohort).apply(attrgetter('n'))

# pivot the df_cohort
cohort_pivot = df_cohort.pivot_table(index = 'cohort',
                                     columns = 'period_number',
                                     values = 'n_customers')

# calculate retention matrix
cohort_size = cohort_pivot.iloc[:,0]
retention_matrix = cohort_pivot.divide(cohort_size, axis = 0)


# plot the retenton matrix
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(12, 8), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(retention_matrix, 
                mask=retention_matrix.isnull(), 
                annot=True, 
                fmt='.0%', 
                cmap='RdYlGn', 
                ax=ax[1])
    ax[1].set_title('Monthly Cohorts: User Retention in Netherlands', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # cohort size
    cohort_size_df = pd.DataFrame(cohort_size).rename(columns={0: 'cohort_size'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_df, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap=white_cmap, 
                ax=ax[0])

    fig.tight_layout()

In [ ]:
plt.plot(retention_matrix.mean())
plt.xlabel('# of periods')
plt.ylabel('retention rate')
plt.title('retention rate in the Netherlands')

### Iceland

In [ ]:
# filtering records for Iceland only
dfIceland = data[data['Country']=='Iceland']

# drop irrelevant columns and duplicate records
dfIceland = dfIceland[['CustomerID', 'InvoiceNo', 'InvoiceDate']].drop_duplicates()

# order month
dfIceland['order_month'] = dfIceland['InvoiceDate'].dt.to_period('M')

# first purchase date for that customer
dfIceland['cohort'] = dfIceland.groupby('CustomerID')['InvoiceDate'] \
                 .transform('min') \
                 .dt.to_period('M')

# a dataframe with cohort, order_month, n_customers, and period number
df_cohort = dfIceland.groupby(['cohort', 'order_month']) \
              .agg(n_customers=('CustomerID', 'nunique')) \
              .reset_index(drop=False)

df_cohort['period_number'] = (df_cohort.order_month - df_cohort.cohort).apply(attrgetter('n'))

# pivot the df_cohort
cohort_pivot = df_cohort.pivot_table(index = 'cohort',
                                     columns = 'period_number',
                                     values = 'n_customers')

# calculate retention matrix
cohort_size = cohort_pivot.iloc[:,0]
retention_matrix = cohort_pivot.divide(cohort_size, axis = 0)


# plot the retenton matrix
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(12, 8), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(retention_matrix, 
                mask=retention_matrix.isnull(), 
                annot=True, 
                fmt='.0%', 
                cmap='RdYlGn', 
                ax=ax[1])
    ax[1].set_title('Monthly Cohorts: User Retention in Iceland', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # cohort size
    cohort_size_df = pd.DataFrame(cohort_size).rename(columns={0: 'cohort_size'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_df, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap=white_cmap, 
                ax=ax[0])

    fig.tight_layout()

In [ ]:
plt.plot(retention_matrix.mean())
plt.xlabel('# of periods')
plt.ylabel('retention rate')
plt.title('retention rate in Iceland')

### EIRE

In [ ]:
# filtering records for EIRE only
dfEIRE = data[data['Country']=='EIRE']

# drop irrelevant columns and duplicate records
dfEIRE = dfEIRE[['CustomerID', 'InvoiceNo', 'InvoiceDate']].drop_duplicates()

# order month
dfEIRE['order_month'] = dfEIRE['InvoiceDate'].dt.to_period('M')

# first purchase date for that customer
dfEIRE['cohort'] = dfEIRE.groupby('CustomerID')['InvoiceDate'] \
                 .transform('min') \
                 .dt.to_period('M')

# a dataframe with cohort, order_month, n_customers, and period number
df_cohort = dfEIRE.groupby(['cohort', 'order_month']) \
              .agg(n_customers=('CustomerID', 'nunique')) \
              .reset_index(drop=False)

df_cohort['period_number'] = (df_cohort.order_month - df_cohort.cohort).apply(attrgetter('n'))

# pivot the df_cohort
cohort_pivot = df_cohort.pivot_table(index = 'cohort',
                                     columns = 'period_number',
                                     values = 'n_customers')

# calculate retention matrix
cohort_size = cohort_pivot.iloc[:,0]
retention_matrix = cohort_pivot.divide(cohort_size, axis = 0)


# plot the retenton matrix
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(12, 8), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(retention_matrix, 
                mask=retention_matrix.isnull(), 
                annot=True, 
                fmt='.0%', 
                cmap='RdYlGn', 
                ax=ax[1])
    ax[1].set_title('Monthly Cohorts: User Retention in EIRE', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # cohort size
    cohort_size_df = pd.DataFrame(cohort_size).rename(columns={0: 'cohort_size'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_df, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap=white_cmap, 
                ax=ax[0])

    fig.tight_layout()

In [ ]:
plt.plot(retention_matrix.mean())
plt.xlabel('# of periods')
plt.ylabel('retention rate')
plt.title('retention rate in EIRE')

## Customer Segmentation using RFM modeling

Before we can proceed, we need a benchmark date from where we can calculate recency i.e. date of analysis. 
Recency=Analysis Date- Transaction date

for the purpose of simplifaction we can assume that analysis commenced 1 day after the last transaction.

In [ ]:
last_txn_date= data["InvoiceDate"].max()
analysis_date= last_txn_date + dt.timedelta(days = 1)

In [ ]:
rfm = data.groupby("CustomerID").agg({"InvoiceNo": [lambda Frequency: Frequency.nunique()],"TotalPrice": lambda TotalMoney: TotalMoney.sum(),"InvoiceDate": [lambda FinalOrder: (analysis_date - FinalOrder.max()).days]})
rfm.head()

In [ ]:
rfm = rfm.reset_index()
rfm

In [ ]:
rfm.columns = ["customer_id", "frequency", "monetary", "recency"]
rfm.head()

In [ ]:
len(rfm)

In [ ]:
rfm['R_score'] = rfm['recency'].rank(ascending=False)
rfm['F_score'] = rfm['frequency'].rank(ascending=True)
rfm['M_score'] = rfm['monetary'].rank(ascending=True)
 
# normalizing the rank of the customers
rfm['R_score_norm'] = (rfm['R_score']/rfm['R_score'].max())*100
rfm['F_score_norm'] = (rfm['F_score']/rfm['F_score'].max())*100
rfm['M_score_norm'] = (rfm['M_score']/rfm['M_score'].max())*100
 
rfm.drop(columns=['R_score', 'F_score', 'M_score'], inplace=True)
 
rfm.head()



For this scenario we can consider the RFM score as the artimetic mean of R,F and M Scores for simplicity, assuming all 3 to have equal importance.

In [ ]:
rfm['Final Score']= round((rfm['R_score_norm']+rfm['F_score_norm']+rfm['M_score_norm'])/3,2)
rfm.head()

In [ ]:
rfm['Segment']= rfm['Final Score'].apply(lambda x: 'Top Customer' if (x >=90) else ('High Value Customer' if (x >=75 and x<90) else('Medium Value Customer' if (x >=50 and x<75) else('Low Value Customer' if (x >=30 and x<50) else 'Lost Customer'))))
rfm.head()

In [ ]:
plt.figure(figsize=(15,7))
sns.countplot(rfm['Segment'])
plt.xlabel("Number of Customers by category")
plt.show()